# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096964


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:03<00:43,  1.50s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:53,  1.91s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<00:37,  1.38s/it]

Rendering models:  16%|█▌        | 5/31 [00:06<00:25,  1.00it/s]

...

Rendering models:  84%|████████▍ | 26/31 [00:10<00:00,  7.51it/s]

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  7.56it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  7.97it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  9.81it/s]

Rocco420                              0.001248
Terrance_L._Johnson                   0.002826
RaeneVigil                            0.000340
jnarayanbvg                           0.001987
not-logged-in-a0d39065895c84f0fd2d    0.000389
TSM_Ginga                             0.001568
Poupoussin                            0.000829
rmz4160                               0.000293
rmz4160                               0.000300
Nauter                                0.000285
jdebn537                              0.000970
insert_name                           0.000192
coyotes54                             0.000257
Brofessoramundsen                     0.000182
not-logged-in-314d48142a3a122ba666    0.000913
fyzxfan                               0.000180
homant                                0.000223
peakscience7                          0.000128
helios1235                            0.000128
Lavadude                              0.020490
clars915                              0.000167
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())